# Pair Programming 
## Lección 15 - ETL III - Transformación II - Clases y funciones de limpieza

En la lección de hoy aprendimos como Crearnos una clase que nos permita limpiar los datos obtenidos de la API.

En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

Happy coding 🦈

In [1]:
import pandas as pd
pd.options.display.max_columns=None
import requests
import ast
import numpy as np

In [3]:
df_ataques = pd.read_csv("attacks_limpieza_completa.csv")

In [175]:
class Etl:
    # constructor
    def __init__(self, producto):
        
        self.producto = producto

    # Método para llamar a la API
    def llamar_api(self, producto, diccionario):
        df_meteo= pd.DataFrame()

        for k,v in diccionario.items():
            df_meteo1 = pd.DataFrame()
            # Los values de latitud y longitud serían = latitud: v[0] y longitud: v[1]. 
            producto = "meteo"
            response = requests.get(url = f'http://www.7timer.info/bin/api.pl?lon=-{v[1]}&lat={v[0]}&product={self.producto}&output=json') 
            df_meteo1 = pd.DataFrame.from_dict(pd.json_normalize(response.json()["dataseries"]))
            df_meteo1["pais"] = k

            df_meteo = pd.concat([df_meteo1, df_meteo], axis=0, ignore_index=True) 
            
            if response.status_code == 200:
                print('La peticion se ha realizado correctamente, se ha devuelto el código de estado:',response.status_code,' y como razón del código de estado: ',response.reason)
            elif response.status_code == 402:
                print('No se ha podido autorizar usario, se ha devuelto el código de estado:', response.status_code,' y como razón del código de estado: ',response.reason)
            elif response.status_code == 404:
                print('Algo ha salido mal, el recurso no se ha encontrado,se ha devuelto el código de estado:', response.status_code,' y como razón del código de estado: ',response.reason)
            else:
                print('Algo inesperado ha ocurrido, se ha devuelto el código de estado:', response.status_code,' y como razón del código de estado: ',response.reason) 
        
        print(f"df_meteo tiene un formato de (filas,columnas): {df_meteo.shape}")
        return df_meteo
        

    def limpiar_rh_profile(self,df_meteo):
        # Función que limpia la columna rh_profile del dataframe
       
        # Resetear el índice del dataframe para no tener columnas repetidas
        df_meteo.reset_index(inplace=True)
        # Convertimos a serie la columna que en su interior tiene un diccionario
        df_rh = df_meteo["rh_profile"].apply(pd.Series)

        # Creamos un dataframe vacío para insertar cada uno de los values del diccionario
        df_rh_profile = pd.DataFrame()
        for i in range(len(df_rh.columns)): 
            #aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
            nombre = "rh_" + str(df_rh[i].apply(pd.Series)["layer"][0]) 
            #hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
            valores = list(df_rh[i].apply(pd.Series)["rh"] )
            #usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
            df_rh_profile.insert(i, nombre, valores)
        
        print(f"df_rh_profile tiene un formato de (filas,columnas): {df_rh_profile.shape}")
        return df_rh_profile

    def limpiar_wind_direction_profile(self, df_meteo):
        # Función que limpia la columna wind_profile del dataframe

        # Resetear el índice del dataframe para no tener columnas repetidas
        df_meteo.reset_index(inplace=True)        
        # Convertimos a serie la columna que en su interior tiene un diccionario
        df_wind = df_meteo["wind_profile"].apply(pd.Series)

        # Creamos un dataframe vacío para insertar cada uno de los values del diccionario
        df_wind_direction_profile = pd.DataFrame()
        for i in range(len(df_wind.columns)): 
            #aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
            nombre = "wind_direction_" + str(df_wind[i].apply(pd.Series)["layer"][0]) 
            #hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
            valores = list(df_wind[i].apply(pd.Series)["direction"])
            #usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
            df_wind_direction_profile.insert(i, nombre, valores)
        print(f"df_wind_direction_profile tiene un formato de (filas,columnas): {df_wind_direction_profile.shape}")
        return df_wind_direction_profile
    
    def limpiar_wind_speed_profile(self, df_meteo):
        
        # Convertimos a serie la columna que en su interior tiene un diccionario
        df_wind = df_meteo["wind_profile"].apply(pd.Series)
        
        # Creamos un dataframe vacío para insertar cada uno de los values del diccionario
        df_wind_speed_profile = pd.DataFrame()
        for i in range(len(df_wind.columns)): 
            #aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
            nombre = "wind_speed_" + str(df_wind[i].apply(pd.Series)["layer"][0]) 
            #hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
            valores = list(df_wind[i].apply(pd.Series)["speed"])
            #usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
            df_wind_speed_profile.insert(i, nombre, valores)
        
        print(f"df_wind_speed_profile tiene un formato de (filas,columnas): {df_wind_speed_profile.shape}")
        return df_wind_speed_profile

    def concatenar_columnas_profile(self, df_rh_profile, df_wind_direction_profile, df_wind_speed_profile):
        df_profile = pd.concat([df_wind_direction_profile, df_wind_speed_profile, df_rh_profile], axis=1)
        print(f"df_profile tiene una formato de (filas,columnas): {df_profile.shape}")
        return df_profile

    def eliminar_columnas_profile(self, df_meteo):
        df_meteo = df_meteo.drop(["rh_profile", "wind_profile"], axis=1)
        print(f"Ahora df_meteo tiene una formato de (filas,columnas): {df_meteo.shape}")
        return df_meteo

    def unir_df_meteo_profile(self, df_meteo, df_profile):
        df_meteo_completo = df_meteo.join(df_profile, how="inner", on="index")
        print(f"Ahora df_meteo_completo tiene una formato de (filas,columnas): {df_meteo_completo.shape}")
        return df_meteo_completo

    def agrupar_paises_media(self, df_meteo_completo):
        df_grupo = df_meteo_completo.groupby("pais")[df_meteo_completo.columns].mean().reset_index()
        print(f"df_grupo tiene una formato de (filas,columnas): {df_grupo.shape}")
        return df_grupo

    def unir_df_grupo_ataques(self, df_grupo, df_ataques):
        df_completo = df_ataques.merge(df_grupo, how="inner", right_on="pais", left_on="country")
        print(f"Ahora df_completo tiene una formato de (filas,columnas): {df_completo.shape}")
        return df_completo

#### Instancia para la clase Etl

In [176]:
etl = Etl("meteo")

#### Llamada a la función "llamar_api"

In [177]:
diccionario_paises = {"usa":[39.7837304, -100.445882], "australia":[-24.7761086, 134.755], "south africa":[-28.8166236, 24.991639], "new zealand":[-41.5000831, 172.8344077], "papua new guinea":[-5.6816069, 144.2489081]}

In [178]:
df_meteo = etl.llamar_api("meteo", diccionario_paises)

La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
df_meteo tiene un formato de (filas,columnas): (320, 17)


In [179]:
df_meteo.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 13}, {'layer': '900m...","[{'layer': '950mb', 'direction': 100, 'speed':...",26,2,10,1012,none,0,0,110,3,papua new guinea
1,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 95, 'speed': ...",26,-1,11,1010,none,0,0,90,3,papua new guinea


#### Llamada a la función que limpia la columna rh_profile

In [180]:
df_rh_profile = etl.limpiar_rh_profile(df_meteo)

df_rh_profile tiene un formato de (filas,columnas): (320, 16)


In [181]:
df_rh_profile.head(2)

,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb
0,13,11,8,7,4,0,-3,-2,-4,-4,-3,0,-2,1,4,-1
1,14,12,7,8,5,-1,-3,-2,-3,-4,-3,-2,-1,0,-2,0


#### Llamada a las funciones que limpian las columnas direction y speed de wind_profile

In [182]:
df_wind_direction_profile = etl.limpiar_wind_direction_profile(df_meteo)

df_wind_direction_profile tiene un formato de (filas,columnas): (320, 16)


In [183]:
df_wind_direction_profile.head(2)

,wind_direction_950mb,wind_direction_900mb,wind_direction_850mb,wind_direction_800mb,wind_direction_750mb,wind_direction_700mb,wind_direction_650mb,wind_direction_600mb,wind_direction_550mb,wind_direction_500mb,wind_direction_450mb,wind_direction_400mb,wind_direction_350mb,wind_direction_300mb,wind_direction_250mb,wind_direction_200mb
0,100,90,85,85,85,80,50,15,295,305,315,280,175,160,175,195
1,95,90,85,85,90,95,60,15,295,295,300,275,145,170,170,195


In [184]:
df_wind_speed_profile = etl.limpiar_wind_speed_profile(df_meteo)

df_wind_speed_profile tiene un formato de (filas,columnas): (320, 16)


In [185]:
df_wind_speed_profile.head(2)

,wind_speed_950mb,wind_speed_900mb,wind_speed_850mb,wind_speed_800mb,wind_speed_750mb,wind_speed_700mb,wind_speed_650mb,wind_speed_600mb,wind_speed_550mb,wind_speed_500mb,wind_speed_450mb,wind_speed_400mb,wind_speed_350mb,wind_speed_300mb,wind_speed_250mb,wind_speed_200mb
0,3,4,4,4,4,3,3,3,3,3,3,3,3,4,5,5
1,3,4,4,4,3,3,3,3,3,3,3,3,2,3,5,5


#### Llamada a la función que concatena las columnas profile en un solo dataframe

In [186]:
df_profile = etl.concatenar_columnas_profile(df_rh_profile, df_wind_direction_profile, df_wind_speed_profile)

df_profile tiene una formato de (filas,columnas): (320, 48)


In [187]:
df_profile.head(2)

,wind_direction_950mb,wind_direction_900mb,wind_direction_850mb,wind_direction_800mb,wind_direction_750mb,wind_direction_700mb,wind_direction_650mb,wind_direction_600mb,wind_direction_550mb,wind_direction_500mb,wind_direction_450mb,wind_direction_400mb,wind_direction_350mb,wind_direction_300mb,wind_direction_250mb,wind_direction_200mb,wind_speed_950mb,wind_speed_900mb,wind_speed_850mb,wind_speed_800mb,wind_speed_750mb,wind_speed_700mb,wind_speed_650mb,wind_speed_600mb,wind_speed_550mb,wind_speed_500mb,wind_speed_450mb,wind_speed_400mb,wind_speed_350mb,wind_speed_300mb,wind_speed_250mb,wind_speed_200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb
0,100,90,85,85,85,80,50,15,295,305,315,280,175,160,175,195,3,4,4,4,4,3,3,3,3,3,3,3,3,4,5,5,13,11,8,7,4,0,-3,-2,-4,-4,-3,0,-2,1,4,-1
1,95,90,85,85,90,95,60,15,295,295,300,275,145,170,170,195,3,4,4,4,3,3,3,3,3,3,3,3,2,3,5,5,14,12,7,8,5,-1,-3,-2,-3,-4,-3,-2,-1,0,-2,0


#### Llamada a la función que elimina las columnas profile en el dataframe meteo

In [188]:
df_meteo = etl.eliminar_columnas_profile(df_meteo)

Ahora df_meteo tiene una formato de (filas,columnas): (320, 17)


In [189]:
df_meteo.head(2)

,level_0,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,0,0,3,1,-9999,-9999,-9999,26,2,10,1012,none,0,0,110,3,papua new guinea
1,1,1,6,1,-9999,-9999,-9999,26,-1,11,1010,none,0,0,90,3,papua new guinea


#### Llamada a la función que une el dataframe meteo y profile en un nuevo dataframe

In [190]:
df_meteo_completo = etl.unir_df_meteo_profile(df_meteo, df_profile)

Ahora df_meteo_completo tiene una formato de (filas,columnas): (320, 65)


In [191]:
df_meteo_completo.head(2)

,level_0,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais,wind_direction_950mb,wind_direction_900mb,wind_direction_850mb,wind_direction_800mb,wind_direction_750mb,wind_direction_700mb,wind_direction_650mb,wind_direction_600mb,wind_direction_550mb,wind_direction_500mb,wind_direction_450mb,wind_direction_400mb,wind_direction_350mb,wind_direction_300mb,wind_direction_250mb,wind_direction_200mb,wind_speed_950mb,wind_speed_900mb,wind_speed_850mb,wind_speed_800mb,wind_speed_750mb,wind_speed_700mb,wind_speed_650mb,wind_speed_600mb,wind_speed_550mb,wind_speed_500mb,wind_speed_450mb,wind_speed_400mb,wind_speed_350mb,wind_speed_300mb,wind_speed_250mb,wind_speed_200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb
0,0,0,3,1,-9999,-9999,-9999,26,2,10,1012,none,0,0,110,3,papua new guinea,100,90,85,85,85,80,50,15,295,305,315,280,175,160,175,195,3,4,4,4,4,3,3,3,3,3,3,3,3,4,5,5,13,11,8,7,4,0,-3,-2,-4,-4,-3,0,-2,1,4,-1
1,1,1,6,1,-9999,-9999,-9999,26,-1,11,1010,none,0,0,90,3,papua new guinea,95,90,85,85,90,95,60,15,295,295,300,275,145,170,170,195,3,4,4,4,3,3,3,3,3,3,3,3,2,3,5,5,14,12,7,8,5,-1,-3,-2,-3,-4,-3,-2,-1,0,-2,0


#### Llamada a la función que agrupa los países en el dataframe meteo_completo

In [192]:
df_grupo = etl.agrupar_paises_media(df_meteo_completo)

df_grupo tiene una formato de (filas,columnas): (5, 63)


/tmp/ipykernel_1198/4159316417.py:109: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_grupo = df_meteo_completo.groupby("pais")[df_meteo_completo.columns].mean().reset_index()


In [193]:
df_grupo.head()

,pais,level_0,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.speed,wind_direction_950mb,wind_direction_900mb,wind_direction_850mb,wind_direction_800mb,wind_direction_750mb,wind_direction_700mb,wind_direction_650mb,wind_direction_600mb,wind_direction_550mb,wind_direction_500mb,wind_direction_450mb,wind_direction_400mb,wind_direction_350mb,wind_direction_300mb,wind_direction_250mb,wind_direction_200mb,wind_speed_950mb,wind_speed_900mb,wind_speed_850mb,wind_speed_800mb,wind_speed_750mb,wind_speed_700mb,wind_speed_650mb,wind_speed_600mb,wind_speed_550mb,wind_speed_500mb,wind_speed_450mb,wind_speed_400mb,wind_speed_350mb,wind_speed_300mb,wind_speed_250mb,wind_speed_200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb
0,australia,223.5,223.5,97.5,3.406250,-9999.0,-9999.0,-9999.0,25.843750,-3.453125,10.750000,1015.546875,3.062500,0.0,3.250000,105.546875,123.671875,133.593750,129.453125,120.000000,110.703125,118.203125,120.703125,108.515625,109.062500,119.375000,107.421875,109.062500,124.453125,134.687500,143.359375,3.515625,3.531250,3.453125,3.390625,3.328125,3.312500,3.359375,3.468750,3.484375,3.562500,3.671875,3.734375,3.890625,4.234375,4.859375,5.296875,13.703125,11.671875,8.562500,6.421875,5.546875,4.859375,4.125000,3.890625,3.328125,3.312500,3.046875,2.812500,2.000000,3.406250,3.640625,5.562500
1,new zealand,95.5,95.5,97.5,5.562500,-9999.0,-9999.0,-9999.0,15.000000,9.890625,10.125000,1019.906250,2.953125,0.0,3.031250,147.109375,160.625000,168.125000,170.703125,173.359375,174.687500,175.937500,190.546875,204.453125,247.734375,267.578125,261.250000,257.500000,253.671875,244.687500,257.890625,3.265625,3.203125,3.187500,3.156250,3.140625,3.250000,3.218750,3.359375,3.437500,3.578125,3.796875,4.140625,4.671875,5.250000,6.234375,6.703125,12.843750,12.390625,4.515625,1.828125,1.125000,1.031250,1.015625,2.421875,2.562500,1.593750,1.812500,2.390625,4.250000,4.578125,2.109375,-0.453125
2,papua new guinea,31.5,31.5,97.5,4.343750,-9999.0,-9999.0,-9999.0,25.984375,-1.046875,11.046875,1010.203125,2.015625,0.0,3.281250,83.671875,82.187500,82.500000,82.968750,81.250000,88.984375,198.984375,243.359375,262.421875,264.921875,263.046875,250.234375,240.859375,223.125000,208.281250,222.578125,3.765625,4.375000,4.437500,4.140625,3.359375,2.453125,2.359375,2.921875,3.625000,4.046875,4.421875,4.421875,4.406250,4.671875,5.265625,6.078125,13.578125,12.140625,8.578125,5.718750,2.781250,0.406250,0.390625,-0.484375,-1.171875,-0.609375,1.968750,2.468750,3.093750,4.250000,5.046875,1.593750
3,south africa,159.5,159.5,97.5,5.390625,-9999.0,-9999.0,-9999.0,23.875000,2.359375,10.203125,1019.281250,0.906250,0.0,2.531250,141.171875,160.000000,170.468750,162.187500,181.718750,194.765625,207.578125,212.890625,215.390625,231.640625,231.640625,236.484375,233.593750,210.078125,216.328125,200.468750,2.515625,2.328125,2.281250,2.171875,2.203125,2.218750,2.343750,2.468750,2.625000,3.125000,3.656250,3.796875,3.718750,3.984375,4.234375,4.687500,12.906250,9.984375,7.015625,7.656250,8.921875,7.937500,8.875000,7.125000,4.265625,2.484375,0.578125,0.453125,2.046875,3.750000,5.468750,8.234375
4,usa,287.5,287.5,97.5,4.500000,-9999.0,-9999.0,-9999.0,11.906250,11.390625,3.921875,1018.125000,0.281250,0.0,2.578125,243.593750,240.703125,263.984375,266.640625,262.109375,254.843750,254.375000,249.140625,266.250000,251.953125,237.421875,241.093750,240.234375,218.593750,222.187500,265.546875,2.703125,2.968750,3.234375,3.578125,3.609375,3.828125,4.203125,4.500000,4.703125,5.093750,5.281250,5.515625,5.906250,6.140625,6.546875,6.406250,3.984375,4.390625,4.781250,4.390625,4.281250,4.437500,3.531250,2.437500,2.937500,3.593750,4.234375,4.031250,4.234375,5.734375,6.671875,4.593750


#### Llamada a la función que une los dataframe meteo_completo y ataques

In [194]:
df_completo = etl.unir_df_grupo_ataques(df_grupo, df_ataques)

Ahora df_completo tiene una formato de (filas,columnas): (4335, 85)


In [195]:
df_completo.head()

,Unnamed: 0,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM,pais,level_0,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.speed,wind_direction_950mb,wind_direction_900mb,wind_direction_850mb,wind_direction_800mb,wind_direction_750mb,wind_direction_700mb,wind_direction_650mb,wind_direction_600mb,wind_direction_550mb,wind_direction_500mb,wind_direction_450mb,wind_direction_400mb,wind_direction_350mb,wind_direction_300mb,wind_direction_250mb,wind_direction_200mb,wind_speed_950mb,wind_speed_900mb,wind_speed_850mb,wind_speed_800mb,wind_speed_750mb,wind_speed_700mb,wind_speed_650mb,wind_speed_600mb,wind_speed_550mb,wind_speed_500mb,wind_speed_450mb,wind_speed_400mb,wind_speed_350mb,wind_speed_300mb,wind_speed_250mb,wind_speed_200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb
0,0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327,usa,287.5,287.5,97.5,4.5,-9999.0,-9999.0,-9999.0,11.90625,11.390625,3.921875,1018.125,0.28125,0.0,2.578125,243.59375,240.703125,263.984375,266.640625,262.109375,254.84375,254.375,249.140625,266.25,251.953125,237.421875,241.09375,240.234375,218.59375,222.1875,265.546875,2.703125,2.96875,3.234375,3.578125,3.609375,3.828125,4.203125,4.5,4.703125,5.09375,5.28125,5.515625,5.90625,6.140625,6.546875,6.40625,3.984375,4.390625,4.78125,4.390625,4.28125,4.4375,3.53125,2.4375,2.9375,3.59375,4.234375,4.03125,4.234375,5.734375,6.671875,4.59375
1,1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,3,7,7,3.804860,usa,287.5,287.5,97.5,4.5,-9999.0,-9999.0,-9999.0,11.90625,11.390625,3.921875,1018.125,0.28125,0.0,2.578125,243.59375,240.703125,263.984375,266.640625,262.109375,254.84375,254.375,249.140625,266.25,251.953125,237.421875,241.09375,240.234375,218.59375,222.1875,265.546875,2.703125,2.96875,3.234375,3.578125,3.609375,3.828125,4.203125,4.5,4.703125,5.09375,5.28125,5.515625,5.90625,6.140625,6.546875,6.40625,3.984375,4.390625,4.78125,4.390625,4.28125,4.4375,3.53125,2.4375,2.9375,3.59375,4.234375,4.03125,4.234375,5.734375,6.671875,4.59375
2,2,2018,Invalid,usa,48.0,Unespecific,Jun,N,M,39.78373,-100.445882,HN,1,0,0,1,0,0,3,7,3,8.403629,usa,287.5,287.5,97.5,4.5,-9999.0,-9999.0,-9999.0,11.90625,11.390625,3.921875,1018.125,0.28125,0.0,2.578125,243.59375,240.703125,263.984375,266.640625,262.109375,254.84375,254.375,249.140625,266.25,251.953125,237.421875,241.09375,240.234375,218.59375,222.1875,265.546875,2.703125,2.96875,3.234375,3.578125,3.609375,3.828125,4.203125,4.5,4.703125,5.09375,5.28125,5.515625,5.90625,6.140625,6.546875,6.40625,3.984375,4.390625,4.78125,4.390625,4.28125,4.4375,3.53125,2.4375,2.9375,3.59375,4.234375,4.03125,4.234375,5.734375,6.671875,4.59375
3,7,2018,Unprovoked,usa,52.0,Lemon shark,May,N,M,39.78373,-100.445882,HN,1,0,0,1,0,0,1,9,7,8.730109,usa,287.5,287.5,97.5,4.5,-9999.0,-9999.0,-9999.0,11.90625,11.390625,3.921875,1018.125,0.28125,0.0,2.578125,243.59375,240.703125,263.984375,266.640625,262.109375,254.84375,254.375,249.140625,266.25,251.953125,237.421875,241.09375,240.234375,218.59375,222.1875,265.546875,2.703125,2.96875,3.234375,3.578125,3.609375,3.828125,4.203125,4.5,4.703125,5.09375,5.28125,5.515625,5.90625,6.140625,6.546875,6.40625,3.984375,4.390625,4.78125,4.390625,4.28125,4.4375,3.53125,2.4375,2.9375,3.59375,4.234375,4.03125,4.234375,5.734375,6.671875,4.59375
4,8,2018,Unprovoked,usa,15.0,Lemon shark,May,N,M,39.78373,-100.445882,HN,1,0,0,1,0,0,1,9,7,4.581659,usa,287.5,287.5,97.5,4.5,-9999.0,-9999.0,-9999.0,11.90625,11.390625,3.921875,1018.125,0.28125,0.0,2.578125,243.59375,240.703125,263.984375,266.640625,262.109375,254.84375,254.375,249.140625,266.25,251.953125,237.421875,241.09375,240.234375,218.59375,222.1875,265.5468